# To Add

- base room .xml à ajouter au scene de Sionna
- accéder aux SceneObject depuis les AssetObject

# Introduction to Sionna RT - Scene and Assets Configuration Tools

In this notebook, you will
- Learn how to programaticaly append or remove assets object to a scene (without using Blender).
- See the impact of the assets within the scene w.r.t. ray-traced channels for link-level simulations instead of stochastic channel models.


## Table of Contents
* [Information On Assets](#Information-On-Assets)
*

## Information On Assets

It is usefull to be able to construct a scene dynamically by adding and removing objects, referred to as assets, thus being able to automatically generate datasets from the scenes and/or define complex scenario. To this end, we propose a few novel functionalities. One can define a scene, e.g. using Blender, import the scene within Sionna and then add new assets to that scene. The assets can also be separately defined in Blender for instance. As an example, one could define a scene with a car park, and append to that scene a varing number of cars to see the impact in terms of RF propagation.


## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [ ]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility


# To Remove

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, AssetObject

# For link-level simulations
from sionna.channel import cir_to_time_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement


## Loading Scenes

The Sionna RT module can either load external scene files (in Mitsuba's XML file format) or it can load one of the [integrated scenes](https://nvlabs.github.io/sionna/api/rt.html#example-scenes).

In this example, we load a scene containing a floor and a basic wall above it. The scene can be seen as a the static reference, on which one can add or remove assets. 

In [ ]:
# Load scene
scene = load_scene(filename="./sionna/rt/scenes/floor_wall/floor_wall.xml")

if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512);
    raise ExitCell
scene.preview()

## Load and Handle Assets

In a static scene, one can add assets at various position to evaluate different scenario without having to design a dedicated scene beforehand. It should be noted, that assets are added to the scene by recreating the scene from scratch. Hence, the action to add or remove an asset object to a scene is not differentiable, but the previous and the novel scenes preserve their differentiability properties as any scene in Sionna. This tool must be seen as a way to programmatically construct the scene, e.g. to generate many random scenes to construct a dataset for neural networks training.

Some assets are located in ./sionna/rt/assets. Let's see how to load and use them.

NB: As a workaround to maintain differentiability when placing an object, one could use the mobility tools from Sionna to move pre-existing objects of the scene into or out of the zone of interest of the scene. Yet, this would require to have put these objects in the scene in the first place.



In [ ]:
# Load an asset
asset = AssetObject(name="asset_0", filename="./sionna/rt/assets/body/body.xml")

# Add the asset to the scene
scene.add(asset)

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

As it can be seen in the preview, a body has been added to the previous scene, by creating a novel scene from scratch.

The body can be handle in the same way as any object of the scene.

It is listed in the .asset_objects method from the scene.

In [ ]:
# Get the assets dictionnary from the scene
print(scene.asset_objects)

The dictionnary contains all the assets from the scene.

Our body "asset_0" is listed as an AssetObject. They are slithly different from others Sionna's SceneObject, as we will demonstrate in this notebook. 

First of all, AssetObject can be composed of several SceneObject. Use the method .shapes to display them.

In [ ]:
# Get the asset object by name as any object from the scene
body_asset = scene.get('asset_0')

# Print the objects (SceneObject) composing the asset (AssetObject)
print(f"{body_asset.shapes}")

Our asset (AssetObject) "asset_0" is composed of one object (SceneObject) "asset_0-body".

The name of the object is based on the asset name "asset_0" associated with the component name defined in Blender "body".

The SceneObject can be seen in the object dictionnary of the scene.

In [ ]:
# Print a dictionnary containing all the objects of the scene
print(scene.objects)

The scene contains three objects:
- The "floor" that was in the basic scene.
- The "wall" that was also in the basic scene.
- The "body", named "asset_0-body", that we have added.

Hence, the body is seen as a classic SceneObject by Sionna.

In [ ]:
# Get the body object from the scene as any SceneObject
body_object = scene.get('asset_0-body')


print(f"Current body asset position: {body_asset.position.numpy()}")
print(f"Current body asset orientation: {body_asset.orientation.numpy()}")
# print(f"Current body asset velocity: {body_asset.velocity.numpy()}")

print(f"Current body object position: {body_object.position.numpy()}")
print(f"Current body object orientation: {body_object.orientation.numpy()}")
print(f"Current body object velocity: {body_object.velocity.numpy()}")

As it can be seen, the position of the object is not [0.,0.,0.], contrary to the asset one. This is because Sionna creates a virtual hit box around the whole asset and return the position of the barycenter of this box. Hence, this position is somewhere at the center of the body. The asset barycenter is the one defined originally in Blender, at the feet. To witness that behavior, we can simply fix the position of the object to [0.,0.,0.] using the SceneObject property.

In [ ]:
original_body_object_position = body_object.position.numpy()
body_object.position = [0.,0.,0.]

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The barycenter of our asset is now at the coordinates [0.,0.,0.]. But this behavior doesn't really assist the user in configuring the scene. 

In [ ]:
# Change the orientation of the SceneObject
body_object.orientation = [np.pi/4.,np.pi/4.,np.pi/4.]

print(f"Orientation of the object: {body_object.orientation.numpy()}")
print(f"Position of the object: {body_object.position.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

After the change of orientation, the position of the barycenter of the object has change. That is because Sionna's hit box is not rotated, but recomputed with edges parallel to the coordinate reference axis (x,y,z), hence modifying the barycenter position.

That's why we advice to directly manipulate the asset itself, as you will be able to conveniently and consistently manipulate group of objects together. Also, this is more natural, as the origin of the asset is defined by the user through the asset generation in Blender.

That's why modifying the position/orientation/velocity of an object from an asset will not modify the asset's information.

In [ ]:
# Show that the position and orientation of the asset has not been modified
print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

# Position the body at its original position, with original orientation
# First set the orientation and then the position to avoid side effect
body_object.orientation = [0.,0.,0.]
body_object.position = original_body_object_position

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The body is back on its feet! Now let's move to one side of the wall using the asset properties.

In [ ]:
# Move the body to one side of the wall
body_asset.position = [1.,0.,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The body object position has been correctly moved along with the asset one. Now let's change its orientation so it faces the wall.

In [ ]:
# Make the body face the wall
body_asset.orientation = [-np.pi/2.,0.,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The center of the body asset is located at its feet. So we can turn it upside down.

In [ ]:
# Turn the body upside down
body_asset.orientation += [0.,np.pi,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

So now that we have play a little with a single body, we will remove it from the scene and use a two bodies asset.

In [ ]:
# Remove the body asset
scene.remove('asset_0')

print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Load an asset
asset = AssetObject(name="asset_0", filename="./sionna/rt/assets/two_persons/two_persons.xml")

# Add the asset to the scene
scene.add(asset)

print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Our asset is composed of two objects "asset_0-person_1" and "asset_0-person_2", as it can be seen in the objects list of the scene.

We can use the asset properties to move the bodies around.

In [ ]:
# Get assets and objects
bodies_asset = asset
body_1_object = scene.get('asset_0-person_1')
body_2_object = scene.get('asset_0-person_2')

# Move the two bodies to one side of the wall
bodies_asset.position = [1.,0.,0.]

print(f"Position of the asset: {bodies_asset.position.numpy()}")
print(f"Orientation of the asset: {bodies_asset.orientation.numpy()}")

print(f"Position of the person 1 object: {body_1_object.position.numpy()}")
print(f"Orientation of the person 1 object: {body_1_object.orientation.numpy()}") 

print(f"Position of the person 2 object: {body_2_object.position.numpy()}")
print(f"Orientation of the person 2 object: {body_2_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Move the two bodies to each side of the wall
bodies_asset.position = [0.,0.,0.]
bodies_asset.orientation = [np.pi/2.,0.,0.]

print(f"Position of the asset: {bodies_asset.position.numpy()}")
print(f"Orientation of the asset: {bodies_asset.orientation.numpy()}")

print(f"Position of the person 1 object: {body_1_object.position.numpy()}")
print(f"Orientation of the person 1 object: {body_1_object.orientation.numpy()}") 

print(f"Position of the person 2 object: {body_2_object.position.numpy()}")
print(f"Orientation of the person 2 object: {body_2_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Now we will demonstrate that the assets have the same interaction with ray tracing than any other Sionna SceneObject.

We will add a TX/RX couple at each side of the wall.

The bodies objects will be individually moved (again this is not recommanded), so that we increase the space between them and can use them as reflectors.

With only reflection activate, we should be able to see 3 paths: one reflected on the floor and two reflected by the bodies.

In [ ]:
# Place the bodies
bodies_asset.orientation = [0.,0.,0.]
body_1_object.position += [0.,1.5,0.]
body_2_object.position += [0.,-1.5,0.]

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Configure the transmitter and receiver arrays
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

scene.rx_array = scene.tx_array

# Add a transmitter and receiver with equal distance from the center of the surface
# The position is set precisely to get perfect reflexion from the bodies hands
scene.add(Transmitter(name="tx", position=[-3.1,0.,2.1]))
scene.add(Receiver(name="rx", position=[3.1,0.,2.1]))

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Fix the scene frequency
scene.frequency = 2.4e9

# No LOS, one reflection max
paths = scene.compute_paths(los=False, reflection=True, max_depth=1, num_samples=1e6, method='exhaustive')

print(np.abs(paths.a[0,0,0,0,0,:,0].numpy()))
print(paths.tau[0,0,0,:].numpy())

# Open 3D preview (only works in Jupyter notebook)
if colab_compat:
    scene.render("scene-cam-0", paths=paths);
    raise ExitCell
scene.preview(paths=paths)

The bodies provided in the asset are composed of numerous small reflective surfaces. The positions of TX and RX were chosen to obtain reflections on the hands.

NB: To see more paths, one could activate the scattering.

There are exactly 9 paths in the scene:
* The first path in time is reflected on the floor.
* The 8 other paths are reflected on the hands. You can zoom on the preview to see the 8 paths. Because of the symetry of the scene, these paths arrive at (almost) the same time.

In [ ]:
a, tau = paths.cir()
t = tau[0,0,0,:]/1e-9 # Scale to ns
a_abs = np.abs(a)[0,0,0,0,0,:,0]
a_max = np.max(a_abs)

# Add dummy entry at start/end for nicer figure
t = np.concatenate([(0.,), t, (np.max(t)*1.1,)])
a_abs = np.concatenate([(np.nan,), a_abs, (np.nan,)])

# And plot the CIR
plt.figure()
plt.title("Channel impulse response realization")

# plt.stem(t, a_abs)
plt.stem(t, a_abs)
plt.xlim([0, np.max(t)])
plt.ylim([-2e-6, a_max*1.1])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|a|$");

On the CIR, we can see the 8 paths reflected by the hands arrived at the same time, around 2.45 ns. Generating the time channel estimate from the CIR should merge the power of these paths.

In [ ]:
# Compute the frequency response of the channel at frequencies.
sampling_freq = 5e9 # 0.2 ns resolution
l_min = -6
l_max = 20
h_time = cir_to_time_channel(bandwidth=sampling_freq, a=a, tau=tau, l_min=l_min, l_max=l_max, normalize=True)
t_time = np.arange(l_min, (l_max+1), 1) * (1/sampling_freq)

h_time_abs = np.abs(h_time[0,0,0,0,0,0,:].numpy())

# And plot the CIR
plt.figure()
plt.title("Time Channel from CIR")
plt.stem(t_time, h_time_abs)
plt.xlim([0, np.max(t_time)])
# plt.ylim([-2e-6, a_max*1.1])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|h_time|$");

Because of the 8 paths reflected by the hands, most of the energy arrive through these paths.

# Material 

# BSDFs

We can modify the materials of individual objects composing an asset.

In [ ]:
# print(list(scene.radio_materials.keys()))

# Set the material of the first body to brick
body_1_object.radio_material = "itu_brick"

print(f"First body material: {body_1_object.radio_material.name}")
print(f"Second body material: {body_2_object.radio_material.name}")

# Open 3D preview (only works in Jupyter notebook)
if colab_compat:
    scene.render("scene-cam-0", paths=paths);
    raise ExitCell
scene.preview()

In [ ]:
# No LOS, one reflection max
paths = scene.compute_paths(los=False, reflection=True, max_depth=1, num_samples=1e6, method='exhaustive')

print(np.abs(paths.a[0,0,0,0,0,:,0].numpy()))

# Open 3D preview (only works in Jupyter notebook)
if colab_compat:
    scene.render("scene-cam-0", paths=paths);
    raise ExitCell
scene.preview(paths=paths)

From the paths absolute values, we can separate the paths that reflected on the body in concrete, and the one on the body in brick.

## Conclusion and Outlook